# Constants:

In [1]:
SEQ_LEN = 180 #240   # how many past candles to use to predict
CANDLES_SHIFT = 2 #5 # how many candles to shift between sequences
NAME = "pp2_m5_ov40th004p_shift2_seq180"
VALIDATION_PCT = 0.2

# Functions:

sequence split

In [2]:
import numpy as np

def splitDf(df):
    
    res = []
    print("")
    print("splitDf")
    while len(df) >= SEQ_LEN + len(df.columns) -3:
        first = df.head(SEQ_LEN + len(df.columns) -3).copy()
        first.index = np.arange(0, len(first))
        res.append(first)
        df = df.tail(len(df) - CANDLES_SHIFT)
        df.index = np.arange(0, len(df))

    print("-done")
    print("")
    return res

balance

In [3]:
def balance(dfs):
    
    buys = []
    sells = []
    holds = []
    for df in dfs:
        if df.at[len(df)-1, 'target'] == 0:
            sells.append(df)
        elif df.at[len(df)-1, 'target'] == 1:
            buys.append(df)
        else:
          holds.append(df)

    print("before balancing:")
    print("buys:", len(buys), ", sells:", len(sells), ", holds:", len(holds))

    smallest = min(len(buys), len(sells), len(holds))
    buys = buys[:smallest]
    sells = sells[:smallest]
    holds = holds[:smallest]

    dfsBalanced = buys+sells+holds
    return dfsBalanced

preprocessing

In [4]:
from sklearn import preprocessing

def preprocess2(dfs):
    # after sequencing
    #
    # pct.change transform price columns ('low', 'high', 'open', 'close')
    # scale every colum (center mean and unit variance)

    for df in dfs:
        for col in df.columns:
            if col != "target":
                if col != 'quantity_baseUnits' and col != 'hl_percent':
                    df[col] = df[col].pct_change()
                    df.dropna(inplace=True)
                df[col] = preprocessing.scale(df[col].values)
                df.index = np.arange(0, len(df))
    return dfs

sequences

In [5]:
def buildSequences(dfs):
    
    sequences = []
    for df in dfs:
        if(len(df) == SEQ_LEN):
            label = df.at[SEQ_LEN-1, 'target']
            df = df.iloc[:, :-1]
            dfArray = df.values.tolist()
            sequences.append([np.array(dfArray), label])
    
    return sequences

X y split

In [6]:
def xySplit(seqWithTarget):

    X = []
    y = []
    for seq, target in seqWithTarget:
        X.append(seq)
        y.append(target)

    return np.array(X),np.array(y)

# DF manipulation, build training sets:

In [7]:
import pandas as pd
df = pd.read_csv("../historicalData/labeled/HistoricalDataLabeled_BTC_USDT_01072016_01072023_MINUTE_5_ov40_th004p.csv")
df = df[['close', 'hl_percent', 'quantity_baseUnits', 'target']]
df

,close,hl_percent,quantity_baseUnits,target
0,681.84,0.005617,0.488352,0
1,683.40,0.007915,0.239165,0
2,681.95,0.002078,0.103008,0
3,678.86,0.008804,9.084392,0
4,684.39,0.000497,0.236453,0
...,...,...,...,...
736051,30408.88,0.000332,0.091832,0
736052,30413.16,0.000211,0.128622,0
736053,30414.06,0.000341,0.090499,0
736054,30426.45,0.000526,0.205135,0


split into dfs with SEQ_LEN rows

In [8]:
splittedDfs = splitDf(df)
print("num sequences:", len(splittedDfs))


splitDf
-done

num sequences: 367938


In [9]:
# preprocess
splittedDfs = preprocess2(splittedDfs)

seperate training and validation

In [10]:
train_dfs = splittedDfs[:(int(len(splittedDfs) * (1-VALIDATION_PCT)))].copy()
val_dfs = splittedDfs[(int(len(splittedDfs) * (1-VALIDATION_PCT))):].copy()

balance buys/sells/holds

In [11]:
# balance buys, sells, and holds
import random

random.shuffle(train_dfs)
random.shuffle(val_dfs)
train_dfs = balance(train_dfs)
val_dfs = balance(val_dfs)
random.shuffle(train_dfs)
random.shuffle(val_dfs)

before balancing:
buys: 104446 , sells: 101497 , holds: 88407
before balancing:
buys: 24725 , sells: 24935 , holds: 23928


In [13]:
print(len(train_dfs))
print(len(val_dfs))

265221
71784


build sequences

In [14]:
sequencesTraining = buildSequences(train_dfs)
sequencesValidation = buildSequences(val_dfs)
random.shuffle(sequencesTraining)
random.shuffle(sequencesValidation)

In [15]:
print(len(sequencesTraining))
print(len(sequencesValidation))

265221
71784


split sequence from label

In [16]:
train_x, train_y = xySplit(sequencesTraining)
validation_x, validation_y = xySplit(sequencesValidation)

In [17]:
print(type(train_x))
print(train_x.shape)
print(type(validation_x))
print(validation_x.shape)

<class 'numpy.ndarray'>
(265221, 180, 3)
<class 'numpy.ndarray'>
(71784, 180, 3)


In [18]:
import pickle

with open(f'../trainData/{NAME}_train_x.pkl', 'wb') as file:
    pickle.dump(train_x, file)
with open(f'../trainData/{NAME}_train_y.pkl', 'wb') as file:
    pickle.dump(train_y, file)
with open(f'../trainData/{NAME}_validation_x.pkl', 'wb') as file:
    pickle.dump(validation_x, file)
with open(f'../trainData/{NAME}_validation_y.pkl', 'wb') as file:
    pickle.dump(validation_y, file)

In [2]:
import pickle

with open(f"../trainData/{NAME}_train_x.pkl", "rb") as file:
    train_x = pickle.load(file)
with open(f"../trainData/{NAME}_train_y.pkl", "rb") as file:
    train_y = pickle.load(file)
with open(f"../trainData/{NAME}_validation_x.pkl", "rb") as file:
    validation_x = pickle.load(file)
with open(f"../trainData/{NAME}_validation_y.pkl", "rb") as file:
    validation_y = pickle.load(file)

# Model:

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


hyper parameters bounds

In [4]:
EPOCHS = 60

batchSize = 96
layers = 2
nodes = 256
denseNodes = 128

dropOut = 0.8 #0.92 #0.88
rec_dropout = 0
l1l2_reg = 0 #1e-5#1e-3

learningRate = 0.00001
decay = 0

hp_suffix = f"bs({batchSize})_layers({layers})_noded({nodes})_dNodes({denseNodes})_do({int(dropOut*100)}%)_recdo({rec_dropout})_l1l2({l1l2_reg})_lr({learningRate})_decay({decay})"

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import os
from tqdm.keras import TqdmCallback
import datetime
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import regularizers
import json
# model
model = Sequential()

for _ in range(layers-1):
  model.add(LSTM(nodes, 
               activation="tanh", 
               recurrent_activation = 'sigmoid', 
               recurrent_dropout = rec_dropout, 
               unroll = False, 
               use_bias = True, 
               input_shape=(train_x.shape[1:]), 
               return_sequences=True,
               kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
               #bias_regularizer=regularizers.l2(l1l2_reg),
               activity_regularizer=regularizers.l2(l1l2_reg)
               ))
  model.add(Dropout(dropOut))
  #model.add(BatchNormalization())

model.add(LSTM(nodes, 
             activation="tanh", 
             recurrent_activation = 'sigmoid', 
             recurrent_dropout = rec_dropout, 
             unroll = False, 
             use_bias = True, 
             input_shape=(train_x.shape[1:]),
             kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
             #bias_regularizer=regularizers.l2(l1l2_reg),
             activity_regularizer=regularizers.l2(l1l2_reg)
             ))
model.add(Dropout(dropOut))
#model.add(BatchNormalization())

model.add(Dense(denseNodes, 
                activation="relu",
                kernel_regularizer=regularizers.l1_l2(l1=l1l2_reg/10, l2=l1l2_reg),
                #bias_regularizer=regularizers.l2(l1l2_reg), 
                activity_regularizer=regularizers.l2(l1l2_reg)))
model.add(Dropout(dropOut))

model.add(Dense(3, activation="softmax"))



# opt  
opt = tf.keras.optimizers.Adam(lr=learningRate) # decay?


model.compile(loss="sparse_categorical_crossentropy",
              optimizer = opt,
              metrics=["accuracy"])


filename = "" + NAME + "-{epoch:02d}"
filepath = f"models/{filename}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1) # saves every epoch

# Prepare TensorBoard callback
log_dir = "../logs/fit/" + f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# store hyper-parameters
# Path to the JSON file
file_path = "models/hps/hps.json"
# Load the JSON file into a dictionary
with open(file_path, "r") as json_file:
    data = json.load(json_file)
# Add a new key-value pair to the dictionary
data[f"{NAME}" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")] = hp_suffix
# Write the updated dictionary back to the file
with open(file_path, "w") as json_file:
    json.dump(data, json_file, indent=4)




# train
history = model.fit(
  train_x, train_y,
  batch_size = batchSize,
  epochs = EPOCHS,
  validation_data=(validation_x, validation_y),
  callbacks=[checkpoint, TqdmCallback(verbose=0), tensorboard_callback])
  #callbacks=[checkpoint])

c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\Johannes\miniconda3\envs\Quant\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
  0%|          | 0/60 [00:00<?, ?epoch/s]

Epoch 1/60
2761/2763 [============================>.] - ETA: 0s - loss: 1.0871 - accuracy: 0.3726

  2%|▏         | 1/60 [01:05<1:04:02, 65.13s/epoch, loss=1.09, accuracy=0.373, val_loss=1.06, val_accuracy=0.426]

2763/2763 [==============================] - 65s 23ms/step - loss: 1.0871 - accuracy: 0.3726 - val_loss: 1.0567 - val_accuracy: 0.4255
Epoch 2/60
2762/2763 [============================>.] - ETA: 0s - loss: 1.0630 - accuracy: 0.4107

...

 93%|█████████▎| 56/60 [56:24<04:09, 62.29s/epoch, loss=1.01, accuracy=0.477, val_loss=1.02, val_accuracy=0.468]

2763/2763 [==============================] - 63s 23ms/step - loss: 1.0090 - accuracy: 0.4772 - val_loss: 1.0183 - val_accuracy: 0.4677
Epoch 57/60
2762/2763 [============================>.] - ETA: 0s - loss: 1.0089 - accuracy: 0.4773

 95%|█████████▌| 57/60 [57:26<03:05, 61.94s/epoch, loss=1.01, accuracy=0.477, val_loss=1.02, val_accuracy=0.467]

2763/2763 [==============================] - 61s 22ms/step - loss: 1.0089 - accuracy: 0.4773 - val_loss: 1.0189 - val_accuracy: 0.4675
Epoch 58/60
 327/2763 [==>...........................] - ETA: 49s - loss: 1.0074 - accuracy: 0.4783

KeyboardInterrupt: 